<a href="https://colab.research.google.com/github/FujiminoDeliaMary/ArtfullCode/blob/main/TP2_prepa_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import des librairies ##

In [72]:
#Imports librairies de base
import numpy as np
import pandas as pd

#Affichage de Graphes
from IPython.display import Markdown, display, HTML
import matplotlib.pyplot as plt

#Information Mutuelle
from sklearn.feature_selection import mutual_info_regression


# Nouvelle section

## 1. Lecture des données ##

1.1 Charger les données d'apprentissage (Data/passagers.csv) dans un DataFrame nommé train. Afficher 10 exemples.

In [73]:

from google.colab import files
import pandas as pd

train = pd.read_csv('data/passagers.csv')
print(train.head(10));




   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   
5            6         0       3   
6            7         0       1   
7            8         0       3   
8            9         1       3   
9           10         1       2   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   
5                                   Moran, Mr. James    male   NaN      0   
6                            McCarthy, Mr. Timothy J    male  54

1.2 Charger les données d'apprentissage (Data/test.csv) dans un Dataframe nommé test. Afficher 10 exemples.

In [74]:
test = pd.read_csv('data/test.csv')
test.head(10)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


1.3 Afficher les informations des deux DataFrames train et Test.

In [75]:
test.info()
print("\n---------------------------------------------------------------------")
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB

---------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-

1.4 Calculer le taux global de survie

In [76]:

taux_survie = train['Survived'].mean()
print(f"Taux de survie : {taux_survie:.2%}")

Taux de survie : 38.38%


#### Question 1 : Quel Travail voyez-vous pour que les données soient prêtes à faire tourner dans un modèle ?

-> encoder les colonnes sexes
-> combler/supprimer les valeurs nulles de la colonnes age
-> Supprimer les colonnes dérangeantes



#### Question 2 : Quelle différence y-a-t'il entre les données train et les données test ?
-> c'est un dataSet qui a été divisé pour donner les données train et test.



#### Question 3 : Quel est le rôle de ces deux sets de données ?
 -> Les données train sont utilisées pour entrâiner le modèle
 -> Les données de test sont utilisées pour évaluer la performance des données après l'entraînement.



## 2. Analyse des features

2.1 Analyser l'effet de la variable Classe sur le taux de survie, en affichant le % de survie par classe

In [77]:
taux_survie_par_classe = train.groupby('Pclass')['Survived'].mean() * 100
print("Taux de survie par classe :")
print(taux_survie_par_classe)

Taux de survie par classe :
Pclass
1    62.962963
2    47.282609
3    24.236253
Name: Survived, dtype: float64


2.2 Analyser l'effet de la variable Genre (Sex) sur le taux de survie, en affichant le % de survie par classe

In [78]:
taux_survie_par_sexe = train.groupby('Sex')['Survived'].mean() * 100
print("Taux de survie par sexe :")
print(taux_survie_par_sexe)

Taux de survie par sexe :
Sex
female    74.203822
male      18.890815
Name: Survived, dtype: float64


2.3 Calculer une nouvelle colonne FamilySize contenant la taille de la famille, pour les DataFrame train et Test. Afficher pour 10 exemples l'Id du passager, le nom, l'age, le sexe et la taille de la famille.

In [79]:
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1

columns = ['PassengerId', 'Name', 'Age', 'Sex', 'FamilySize']
print("Exemples Train :")
print(train[columns].head(10))

print("\n----------------------------------------------------------------------------------")

print("\nExemples Test :")
print(test[columns].head(10))

Exemples Train :
   PassengerId                                               Name   Age  \
0            1                            Braund, Mr. Owen Harris  22.0   
1            2  Cumings, Mrs. John Bradley (Florence Briggs Th...  38.0   
2            3                             Heikkinen, Miss. Laina  26.0   
3            4       Futrelle, Mrs. Jacques Heath (Lily May Peel)  35.0   
4            5                           Allen, Mr. William Henry  35.0   
5            6                                   Moran, Mr. James   NaN   
6            7                            McCarthy, Mr. Timothy J  54.0   
7            8                     Palsson, Master. Gosta Leonard   2.0   
8            9  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)  27.0   
9           10                Nasser, Mrs. Nicholas (Adele Achem)  14.0   

      Sex  FamilySize  
0    male           2  
1  female           2  
2  female           1  
3  female           2  
4    male           1  
5    male    

2.4 Visualisez le nombre de personnes et le taux de survie par taille de famille.

In [80]:
taux_survie_par_sexe = train.groupby('FamilySize')['Survived'].mean() * 100
print("Taux de survie par sexe :")
print(taux_survie_par_sexe)

Taux de survie par sexe :
FamilySize
1     30.353818
2     55.279503
3     57.843137
4     72.413793
5     20.000000
6     13.636364
7     33.333333
8      0.000000
11     0.000000
Name: Survived, dtype: float64


2.5 Calculer une nouvelle colonne IsAlone indiquant si le passager est seul ou pas, pour les DataFrame train et Test. Afficher pour 10 exemples l'Id du passager, le nom, l'age, le sexe et le status IsAlone.

In [81]:
train['IsAlone'] = train['FamilySize'] - 1 == 0
test['IsAlone'] = test['FamilySize'] - 1 == 0

columns = ['PassengerId', 'Name', 'Age', 'Sex', 'IsAlone']
print("Exemples Train :")
print(train[columns].head(10))

print("\n----------------------------------------------------------------------------------")

print("\nExemples Test :")
print(test[columns].head(10))

Exemples Train :
   PassengerId                                               Name   Age  \
0            1                            Braund, Mr. Owen Harris  22.0   
1            2  Cumings, Mrs. John Bradley (Florence Briggs Th...  38.0   
2            3                             Heikkinen, Miss. Laina  26.0   
3            4       Futrelle, Mrs. Jacques Heath (Lily May Peel)  35.0   
4            5                           Allen, Mr. William Henry  35.0   
5            6                                   Moran, Mr. James   NaN   
6            7                            McCarthy, Mr. Timothy J  54.0   
7            8                     Palsson, Master. Gosta Leonard   2.0   
8            9  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)  27.0   
9           10                Nasser, Mrs. Nicholas (Adele Achem)  14.0   

      Sex  IsAlone  
0    male    False  
1  female    False  
2  female     True  
3  female    False  
4    male     True  
5    male     True  
6    male 

2.6 Visualisez le nombre de personnes et le taux de survie par statut seul ou pas.

In [82]:
taux_survie_par_statut = train.groupby('Survived')['IsAlone'].mean() * 100
print("Taux de survie par statut :")
print(taux_survie_par_statut)





Taux de survie par statut :
Survived
0    68.123862
1    47.660819
Name: IsAlone, dtype: float64


2.7 Calculer l'information mutuelle sur les données train.

Attention : Il y a des données continues et des données discrètes

Ignorer les colonnes Name, Ticket et Cabin

2.8 Afficher dans un graphique les scores d'information mutuelle par feature

### Question 1 : La variable Classe est-elle utile pour prévoir la survie ? Pourquoi ?

Oui, la variable Pclass  est utile pour prévoir la survie. Car les analyses montrent clairement la diférence de taux de survie entre les classes.

### Question 2 : Même question pour la variable Sex

Oui, la variable Sex est très utile pour prévoir la survie. Car grâce à cela, on a pu constater que les femmes ont un taux de survie bien plus élevé que les hommes.

### Question 3 : Comment traiter le variables liées à la famille du passager ?





## 3. Traitement des données manquantes (à faire sur les données Train et Test)

3.1 La variable "Embarked" possède des valeurs nulles, compléter par la valeur majoritaire (S).

In [83]:
#Vérifier les valeurs nulles
print(test['Embarked'].isnull().sum(), "Valeurs nulles dans Embarked pour la variable test")
print(train['Embarked'].isnull().sum(), "Valeurs nulles dans Embarked pour la variable train")

#Remplir les valeurs nulles avec S
train['Embarked'].fillna('S')
train['Embarked'].fillna('S')


0 Valeurs nulles dans Embarked pour la variable test
2 Valeurs nulles dans Embarked pour la variable train


,Embarked
0,S
1,C
2,S
3,S
4,S
...,...
886,S
887,S
888,S
889,C


3.2 Afficher l'influence du port d'embarquement sur la Survie

In [84]:
print(train.groupby('Embarked')['Survived'].mean())


Embarked
C    0.553571
Q    0.389610
S    0.336957
Name: Survived, dtype: float64


3.3 La variable "Fare" (prix du ticket) possède des valeurs manquantes. Remplacer ces valeurs par la valeur médiane.

In [85]:
#Vérifier le nombre de valeurs manquantes de Fare
print(test['Fare'].isnull().sum(), "Valeurs nulles dans Fare pour la variable test")
print(train['Fare'].isnull().sum(), "Valeurs nulles dans Fare pour la variable train")

test.fillna(test['Fare'].median(), inplace=True)
train.fillna(train['Fare'].median(), inplace=True)
print("\n--------------------------------------------------------------------------")
#Vérifier le nombre de valeurs manquantes de Fare
print(test['Fare'].isnull().sum(), "Valeurs nulles dans Fare pour la variable test après")
print(train['Fare'].isnull().sum(), "Valeurs nulles dans Fare pour la variable train après")

1 Valeurs nulles dans Fare pour la variable test
0 Valeurs nulles dans Fare pour la variable train

--------------------------------------------------------------------------
0 Valeurs nulles dans Fare pour la variable test après
0 Valeurs nulles dans Fare pour la variable train après


3.4 Créer une variable CategoricalFare Découpant la valeur de ticket en 4 catégories de tailles à peu près égales.

In [86]:
train['CategoricalFare'] = pd.cut(train['Fare'], 4, labels=['Low', 'Medium', 'High', 'Very High'])
test['CategoricalFare'] = pd.cut(test['Fare'], 4, labels=['Low', 'Medium', 'High', 'Very High'])


\\3.5 Analyser l'effet de ces catégories de prix de ticket sur la survie.

In [87]:
print("Taux de survie par catégorie de prix de ticket :")
print(train.groupby('CategoricalFare')['Survived'].mean())

Taux de survie par catégorie de prix de ticket :
CategoricalFare
Low          0.368113
Medium       0.724138
High         0.666667
Very High    1.000000
Name: Survived, dtype: float64


<ipython-input-87-739b99512a86>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(train.groupby('CategoricalFare')['Survived'].mean())


3.6 Combien de valeurs d'Age manquent ? Complète par des valeurs entières aléatoires entre (moyenne - écart type) et (moyenne + écart type).

In [88]:
print(test['Age'].isnull().sum(), "valeurs d'âges manquantes dans la variable test")
print(train['Age'].isnull().sum(), "valeurs d'âges manquantes dans la variable test")

mean_age = train['Age'].mean()
std_age = train['Age'].std() #écart type

random_ages = np.random.randint(mean_age - std_age, mean_age + std_age)

train.loc[train['Age'].isnull(), 'Age'] = random_ages

0 valeurs d'âges manquantes dans la variable test
0 valeurs d'âges manquantes dans la variable test


3.7 Créer une variable CategoricalAge Découpant la valeur d'age en 5 tranches d'age de même taille.

In [89]:
train['CategoricalAge'] = pd.cut(train['Age'], 5, labels=['Baby', 'Child', 'Teenager', 'Adult', 'Old'])
test['CategoricalAge'] = pd.cut(test['Age'], 5, labels=['Baby', 'Child', 'Teenager', 'Adult', 'Old'])

print(train[['Age', 'CategoricalAge']])

print("\n---------------------------------------")

print(test[['Age', 'CategoricalAge']])

         Age CategoricalAge
0    22.0000          Child
1    38.0000       Teenager
2    26.0000          Child
3    35.0000       Teenager
4    35.0000       Teenager
..       ...            ...
886  27.0000          Child
887  19.0000          Child
888  14.4542           Baby
889  26.0000          Child
890  32.0000          Child

[891 rows x 2 columns]

---------------------------------------
         Age CategoricalAge
0    34.5000       Teenager
1    47.0000          Adult
2    62.0000            Old
3    27.0000          Child
4    22.0000          Child
..       ...            ...
413  14.4542           Baby
414  39.0000       Teenager
415  38.5000       Teenager
416  14.4542           Baby
417  14.4542           Baby

[418 rows x 2 columns]


3.8 Analyser l'effet de ces catégories de prix de ticket sur la survie.

In [90]:
#Déjà répondu à la question 3.4

print(train.groupby('CategoricalAge')['Survived'].mean())


CategoricalAge
Baby        0.386282
Child       0.369942
Teenager    0.404255
Adult       0.434783
Old         0.090909
Name: Survived, dtype: float64


<ipython-input-90-502a5aae2cc1>:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(train.groupby('CategoricalAge')['Survived'].mean())


### Question 1 : Classer les variables par ordre d'intérêt

Par ordre d'intérêt, nous avons
 -> Sex
 -->Fare
 -> Pclass

## 4 : Traitement de features complexes

4.1 : Le nom permet de retrouver le titre du passager. Trouver les titres lorsqu'ils existent.

In [91]:
train['Title'] = train['Name'].str.extract(r',\s*([^\.]+)\.')
test['Title'] = test['Name'].str.extract(r',\s*([^\.]+)\.')

print('Dataframe train :')
print(train['Title'].unique())

print('\n-----------------------------------------------------------------------')

print('Dataframe test :')
print(test['Title'].unique())

Dataframe train :
['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'the Countess' 'Jonkheer']

-----------------------------------------------------------------------
Dataframe test :
['Mr' 'Mrs' 'Miss' 'Master' 'Ms' 'Col' 'Rev' 'Dr' 'Dona']


4.2 : En utilisant la fonction pandas crosstab, afficher pour train le nombre d'occurences par titres (en ligne) et par sexe (en colonne)

In [92]:
print(pd.crosstab(train['Title'], train['Sex']))

print('\n---------------------------------------------------')

print(pd.crosstab(test['Title'], test['Sex']))

Sex           female  male
Title                     
Capt               0     1
Col                0     2
Don                0     1
Dr                 1     6
Jonkheer           0     1
Lady               1     0
Major              0     2
Master             0    40
Miss             182     0
Mlle               2     0
Mme                1     0
Mr                 0   517
Mrs              125     0
Ms                 1     0
Rev                0     6
Sir                0     1
the Countess       1     0

---------------------------------------------------
Sex     female  male
Title               
Col          0     2
Dona         1     0
Dr           0     1
Master       0    21
Miss        78     0
Mr           0   240
Mrs         72     0
Ms           1     0
Rev          0     2


4.3 :
Remplacer 'Melle et 'Ms' par 'Miss'

Remplacer 'Mme' par 'Mrs'

Creer une valeur "Rare" pour les autres titres avec moins de 10 occurences.

Calculer ensuite le taux de survie par titre

In [93]:
#Faire les remplacements
train["Title"] = train["Title"].replace(["Mlle", "Ms"], "Miss")
train["Title"] = train["Title"].replace("Mme", "Mrs")

test["Title"] = test["Title"].replace(["Mlle", "Ms"], "Miss")
test["Title"] = test["Title"].replace("Mme", "Mrs")

#Créer une valeur rare pour les titres avec moins de 10 occurences
#je dois créer une valeur rare ? ou remplacer le titre par "Rare"


#Calculer ensuite le taux de survie par titre
print(train.groupby('Title')['Survived'].mean())

Title
Capt            0.000000
Col             0.500000
Don             0.000000
Dr              0.428571
Jonkheer        0.000000
Lady            1.000000
Major           0.500000
Master          0.575000
Miss            0.702703
Mr              0.156673
Mrs             0.793651
Rev             0.000000
Sir             1.000000
the Countess    1.000000
Name: Survived, dtype: float64


## 5. Mise en forme des données non numériques

5.1 Attribuer une valeur numérique au Genre. Par exemple 0 pour 'female' et 1 pour 'male'

In [94]:
train['Sex'] = train['Sex'].replace({'female': 0, 'male': 1})
test['Sex'] = test['Sex'].replace({'female': 0, 'male': 1})

<ipython-input-94-5041fdbb683d>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['Sex'] = train['Sex'].replace({'female': 0, 'male': 1})
<ipython-input-94-5041fdbb683d>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test['Sex'] = test['Sex'].replace({'female': 0, 'male': 1})


5.2 : Attribuer une valeur pour les possibilités de la variable "Title" ('Master','Miss','Mr','Mrs','Rare')

In [95]:
#Connaître toutes les valeurs possibles de la variable Title
print("Titres uniques dans train :")
print(train['Title'].unique())

print("\nTitres uniques dans test :")
print(test['Title'].unique())

title_mapping = {"Mr": 1,
                 "Miss": 2,
                 "Mrs": 3,
                 "Master": 4,
                 "Rare": 5,
                 "Don": 6,
                 "Rev" :7,
                 "Dona":8,
                 "Jonkheer": 9,
                 "Dr": 10,
                 "Major": 11,
                 "Capt": 12,
                 "Lady": 13,
                 "Sir": 14,
                 "Col": 15,
                 "the Countess" : 16,
                 }
train['Title'] = train['Title'].replace(title_mapping)
test['Title'] = test['Title'].replace(title_mapping)

print(train[['Title']])
print(test[['Title']])

print(train['Title'].unique())


Titres uniques dans train :
['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Major' 'Lady' 'Sir' 'Col'
 'Capt' 'the Countess' 'Jonkheer']

Titres uniques dans test :
['Mr' 'Mrs' 'Miss' 'Master' 'Col' 'Rev' 'Dr' 'Dona']
     Title
0        1
1        3
2        2
3        3
4        1
..     ...
886      7
887      2
888      2
889      1
890      1

[891 rows x 1 columns]
     Title
0        1
1        3
2        1
3        1
4        3
..     ...
413      1
414      8
415      1
416      1
417      4

[418 rows x 1 columns]
[ 1  3  2  4  6  7 10 11 13 14 15 12 16  9]


<ipython-input-95-dc8c509209c8>:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['Title'] = train['Title'].replace(title_mapping)
<ipython-input-95-dc8c509209c8>:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test['Title'] = test['Title'].replace(title_mapping)


5.3 Remplacer les titres manquant par 0, et attribuer à la colonne le type entier

In [96]:

train['Title'].fillna(0)
test['Title'].fillna(0)

train['Title'] = train['Title'].astype(int)
test['Title'] = test['Title'].astype(int)


5.4 Attribuer une valeur numérique aux 4 catégories de la variable Fare

In [97]:
mapping = {'Low': 1, 'Medium': 2, 'High': 3, 'Very High': 4}
train["CategoricalFare"] = train["CategoricalFare"].map(mapping)
test["CategoricalFare"] = test["CategoricalFare"].map(mapping)
print(train["Fare"].head(100))

print(train["Fare"].unique())

0      7.2500
1     71.2833
2      7.9250
3     53.1000
4      8.0500
       ...   
95     8.0500
96    34.6542
97    63.3583
98    23.0000
99    26.0000
Name: Fare, Length: 100, dtype: float64
[  7.25    71.2833   7.925   53.1      8.05     8.4583  51.8625  21.075
  11.1333  30.0708  16.7     26.55    31.275    7.8542  16.      29.125
  13.      18.       7.225   26.       8.0292  35.5     31.3875 263.
   7.8792   7.8958  27.7208 146.5208   7.75    10.5     82.1708  52.
   7.2292  11.2417   9.475   21.      41.5792  15.5     21.6792  17.8
  39.6875   7.8     76.7292  61.9792  27.75    46.9     80.      83.475
  27.9     15.2458   8.1583   8.6625  73.5     14.4542  56.4958   7.65
  29.      12.475    9.       9.5      7.7875  47.1     15.85    34.375
  61.175   20.575   34.6542  63.3583  23.      77.2875   8.6542   7.775
  24.15     9.825   14.4583 247.5208   7.1417  22.3583   6.975    7.05
  14.5     15.0458  26.2833   9.2167  79.2      6.75    11.5     36.75
   7.7958  12.525   66.6 

5.5 Attibuer une valeur numérique au Port d'embarquement (Embarked)

In [98]:
embarked = {'S': 0, 'C': 1, 'Q': 2}
train['Embarked'] = train['Embarked'].map(embarked)
test['Embarked'] = test['Embarked'].map(embarked)

5.6 Supprimer les features inutiles

In [99]:
columns_dropped = ['Name', 'Ticket', 'Cabin']
train.drop(columns=columns_dropped, inplace=True)
test.drop(columns=columns_dropped, inplace=True)



5.7 Afficher les infos pour les deux datasets

In [101]:
print("\nInformations sur le dataset train :")
print(train.info())

print("\nInformations sur le dataset test :")
print(test.info())


Informations sur le dataset train :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   PassengerId      891 non-null    int64   
 1   Survived         891 non-null    int64   
 2   Pclass           891 non-null    int64   
 3   Sex              891 non-null    int64   
 4   Age              891 non-null    float64 
 5   SibSp            891 non-null    int64   
 6   Parch            891 non-null    int64   
 7   Fare             891 non-null    float64 
 8   Embarked         889 non-null    float64 
 9   FamilySize       891 non-null    int64   
 10  IsAlone          891 non-null    bool    
 11  CategoricalFare  891 non-null    category
 12  CategoricalAge   891 non-null    category
 13  Title            891 non-null    int64   
dtypes: bool(1), category(2), float64(3), int64(8)
memory usage: 79.7 KB
None

Informations sur le dataset

5.8 Sauver les deux datasets dans des fichier csv. Eviter d'écrire un index (n° de ligne)

In [102]:
train.to_csv('train_cleaned.csv', index=False)
test.to_csv('test_cleaned.csv', index=False)

print("Datasets sauvegardés et travail terminé")

Datasets sauvegardés et travail terminé
